# Libraries

In [1]:
import pandas as pd
import json
import regex as re
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
import emoji
from sklearn.decomposition import LatentDirichletAllocation
from gensim import corpora, models
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
from underthesea import word_tokenize
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier, plot_importance
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from scipy.sparse import hstack

# Read csv

In [4]:
df_mbb_gp = pd.read_csv('crawl/mbb_gp.csv')
df_mbb_as = pd.read_csv('crawl/mbb_as.csv')

/tmp/ipykernel_33195/2466199199.py:1: DtypeWarning: Columns (6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mbb_gp = pd.read_csv('crawl/mbb_gp.csv')


In [5]:
df_mbb_gp

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,f541ca62-d089-4c45-b01d-b941d5762fa0,Hieu Nguyen,https://play-lh.googleusercontent.com/a-/ALV-U...,App chán thế hả trời updat suốt nhưng lỗi suốt...,1,30,6.2.16,2023-12-26 10:30:47,Chào bạn! MB rất xin lỗi bạn vì trải nghiệm tr...,2022-08-25 09:47:58,6.2.16
1,68dd0200-8bf6-4cff-be94-b477bcb09503,Dan Thien,https://play-lh.googleusercontent.com/a/ACg8oc...,"chán MB thật , lỗi liên tục gửi khiếu nại bên ...",1,16,6.2.16,2023-12-26 11:11:42,Chào bạn! MB rất xin lỗi bạn vì trải nghiệm tr...,2023-12-26 13:09:56,6.2.16
2,6673b26e-b2ea-4453-ac6f-8111c88b478e,Quế Dương,https://play-lh.googleusercontent.com/a/ACg8oc...,"Chán , không cả vào đc cứ lỗi lỗi , ko vào đc",1,10,6.2.16,2023-12-26 15:52:41,Chào bạn! MB rất xin lỗi bạn vì trải nghiệm tr...,2023-12-26 20:11:29,6.2.16
3,7a8cf552-a652-4590-bbc5-ca5a4606ca40,Tình Hồng Mobile,https://play-lh.googleusercontent.com/a-/ALV-U...,1 tuần có7 ngày thì 3 ngày bị lỗi . app cũng c...,1,86,6.2.15,2023-12-23 15:09:45,Chào bạn! MB rất xin lỗi bạn vì trải nghiệm tr...,2023-05-25 17:25:56,6.2.15
4,b14dbb13-11bb-4b28-98a2-c6aebbd68e38,Cường Nguyễn Văn,https://play-lh.googleusercontent.com/a/ACg8oc...,"Tại sao nhấn vào ứng dụng thì lại ""tài khoản g...",1,1,6.2.15,2023-12-26 07:43:31,Chào bạn! MB rất xin lỗi bạn vì trải nghiệm tr...,2023-12-26 13:06:29,6.2.15
...,...,...,...,...,...,...,...,...,...,...,...
113999,4b675e43-d265-48e8-91ed-02a336020eef,Nguyễn Văn Minh,https://play-lh.googleusercontent.com/a/ACg8oc...,........... .........,1,0,NaN,2021-04-22 06:41:13,NaN,NaN,NaN
114000,8434a0ee-0933-4047-ac0b-baed630acfcd,Thư Mai,https://play-lh.googleusercontent.com/a/ACg8oc...,ko bie5 dung 👩????????????????????????????????...,5,0,NaN,2020-12-25 20:17:58,NaN,NaN,NaN
114001,2e36923f-c3b8-4d3b-9853-2167fe401608,Lâm Lữ Văn,https://play-lh.googleusercontent.com/a/ACg8oc...,😘,2,0,NaN,2023-06-22 09:26:32,Cám ơn bạn đã đồng hành cùng MBbank tron...,2023-06-22 10:00:53,NaN
114002,87e4cbd6-43a7-4637-8c97-185fea4798c5,Cường Hoàng,https://play-lh.googleusercontent.com/a/ACg8oc...,🙂,2,0,NaN,2023-12-10 12:42:44,Chào bạn! MB rất xin lỗi bạn vì trải nghiệm tr...,2023-12-09 18:40:44,NaN


In [6]:
df_mbb_as

,date,developerResponse,review,rating,isEdited,userName,title
0,2020-06-05 13:47:05,"{'id': 29078206, 'body': 'Chào bạn, MB rất cảm...",Thích nhất là khi chuyển khoản trên app được m...,5,False,Tu achiho,Mb Bank thật tuyệt
1,2020-02-14 10:58:28,"{'id': 29078262, 'body': 'Chào bạn, MB rất cảm...","dễ sử dụng, bố mẹ mình không rành công nghệ nh...",5,False,bichhuong126,Bichhuong126
2,2020-10-15 16:56:22,"{'id': 29030645, 'body': 'Chào bạn! MB rất xin...","Mình bị mất máy, đăng nhập qua máy khác thì kh...",2,False,110230041994,Không hài hòng với mã OTP
3,2020-06-27 13:23:44,"{'id': 29078220, 'body': 'MB rất xin lỗi về vì...","Đăng ký tk số đẹp 2 ngày chưa xong, điền thông...",1,False,Luke6655,Thất vọng!!!!
4,2020-11-20 09:33:38,"{'id': 29030382, 'body': 'Chào bạn, MB rất xin...","Có ai gặp trường hợp như mình không, vào App ở...",2,False,Linhtkktkk,Vấn đề về load mạng
...,...,...,...,...,...,...,...
19430,2017-08-09 12:54:20,"{'id': 40147731, 'body': 'Chào bạn! MBBank rấ...",Không tiện dụng bằng bankplus,1,False,manh hing,Đánh giá
19431,2017-07-28 03:24:53,"{'id': 40147748, 'body': 'Chào bạn! MBBank rấ...",Là người nhà của MB nhưng tôi vẫn phải đưa ra ...,1,False,PHDoremon,chưa tốt
19432,2017-06-20 02:50:37,"{'id': 40148324, 'body': 'MBBank xin ghi nhận ...",Nên thêm tính năng đăng nhập bằng vân tay như ...,5,False,Ok vkl,Đăng nhập bằng vân tay
19433,2017-06-15 06:13:22,"{'id': 40148346, 'body': 'Chào bạn! MBBank rấ...",Quá tệ. Chắc lại thuê bên Viettel làm app,1,False,Adam R0ck,Không tải được app
